In [1]:
import numpy as np
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange, fetch_movielens
from lightfm.evaluation import precision_at_k, auc_score

from scipy.spatial.distance import cdist

/Users/Fahd_Alhazmi/mambaforge/envs/lightfmtest/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [ ]:
def print_log(row, header=False, spacing=12):
    top = ''
    middle = ''
    bottom = ''
    for r in row:
        top += '+{}'.format('-'*spacing)
        if isinstance(r, str):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, int):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif (isinstance(r, float)
              or isinstance(r, np.float32)
              or isinstance(r, np.float64)):
            middle += '| {0:^{1}.5f} '.format(r, spacing-2)
        bottom += '+{}'.format('='*spacing)
    top += '+'
    middle += '|'
    bottom += '+'
    if header:
        print(top)
        print(middle)
        print(bottom)
    else:
        print(middle)
        print(top)

def patk_learning_curve(model, train, test, eval_train,
                        iterarray, user_features=None,
                        item_features=None, k=5,
                        **fit_params):
    old_epoch = 0
    train_patk = []
    test_patk = []
    headers = ['Epoch', 'train p@{}'.format(k), 'test p@{}'.format(k)]
    print_log(headers, header=True)
    for epoch in iterarray:
        more = epoch - old_epoch
        model.fit_partial(train, 
                          user_features=user_features,
                          item_features=item_features,
                          epochs=more, 
                          **fit_params)
        this_test = precision_at_k(model, test, train_interactions=None, k=k)
        this_train = precision_at_k(model, eval_train, train_interactions=None, k=k)

        train_patk.append(np.mean(this_train))
        test_patk.append(np.mean(this_test))
        row = [epoch, train_patk[-1], test_patk[-1]]
        print_log(row)
    return model, train_patk, test_patk


In [ ]:
eval_train = train

# user_index = np.arange(train.shape[0])
# eval_train = train.copy()
# non_eval_users = list(set(range(train.shape[0])) - set(user_index))
# eval_train = eval_train.tolil()
# for u in non_eval_users:
#     eval_train[u, :] = 0.0
# eval_train = eval_train.tocsr()

In [ ]:
# Instantiate and train the model
model = LightFM(no_components=20, loss='warp', user_alpha=0, item_alpha=0, random_state=42)

In [ ]:
model.fit(train, epochs=0);
iterarray = range(10, 60, 10)
model, train_patk, test_patk = patk_learning_curve(
    model, train, test, eval_train, iterarray, k=10, **{'num_threads': 1}
)


In [ ]:
# from skopt import forest_minimize
# def objective(params):
#     # unpack
#     epochs, learning_rate, no_components, alpha = params
    
#     user_alpha = alpha
#     item_alpha = alpha
#     model = LightFM(loss='warp',
#                     random_state=42,
#                     learning_rate=learning_rate,
#                     no_components=no_components,
#                     user_alpha=user_alpha,
#                     item_alpha=item_alpha)
#     model.fit(train, epochs=epochs, num_threads=4, verbose=True)    
#     patks = precision_at_k(model, test, train_interactions=None, k=5, num_threads=4)
#     mapatk = np.mean(patks)
    
#     # Make negative because we want to _minimize_ objective
#     out = -mapatk
    
#     # Handle some weird numerical shit going on
#     if np.abs(out + 1) < 0.01 or out < -1.0:
#         return 0.0
#     else:
#         return out

# space = [(1, 260), # epochs
#          (10**-4, 1.0, 'log-uniform'), # learning_rate
#          (20, 200), # no_components
#          (10**-6, 10**-1, 'log-uniform'), # alpha
#         ]
# res_fm = forest_minimize(objective, space, n_calls=250,
#                      random_state=0,
#                      verbose=True)
# print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
# print('Optimal parameters:')
# params = ['epochs', 'learning_rate', 'no_components', 'alpha']
# for (p, x_) in zip(params, res_fm.x):
#     print('{}: {}'.format(p, x_))
